In [24]:
#  1，导入需要的库
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import load_boston
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor

In [29]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
#  2,以波士顿数据集违为例，导入完整的数据集并探索
dataset = load_boston()

In [4]:
dataset.data.shape

(506, 13)

In [9]:
x_full,y_full = dataset.data,dataset.target
n_samples = x_full.shape[0]
n_features = x_full.shape[1]

In [10]:
# 3,为完整数据放入缺失值
#首选确定我们希望放入的确实的数据的比例，在这里我们假设是50%，那总共就是有3289个数据缺失
rng = np.random.RandomState(0)
missing_rate = 0.5
n_missing_samples = int(np.floor(n_samples*n_features*missing_rate))

In [11]:
missing_features = rng.randint(0,n_features,n_missing_samples)
missing_samples = rng.randint(0,n_samples,n_missing_samples)

In [12]:
x_missing = x_full.copy()
y_missing = y_full.copy()
x_missing[missing_samples,missing_features] = np.nan
x_full = pd.DataFrame(x_missing)
#转换成df是为了后续方便各种操作

In [13]:
from sklearn.impute import SimpleImputer
imp_mean = SimpleImputer(missing_values = np.nan,strategy="mean")
x_missing_mean =  imp_mean.fit_transform(x_missing)

In [14]:
imp_0 = SimpleImputer(missing_values = np.nan,strategy = "constant",fill_value=0)
x_missing_0 = imp_0.fit_transform(x_missing)

In [15]:
x_missing_reg = pd.DataFrame(x_missing.copy())
np.argsort(x_missing_reg.isnull().sum(axis=0)).values

array([ 6, 12,  8,  7,  9,  0,  2,  1,  5,  4,  3, 10, 11], dtype=int64)

In [11]:
df = x_missing_reg
df.shape

(506, 13)

In [16]:
for i in list(range(0,12)):
    df = x_missing_reg
    fillc = df.iloc[:,i]
    df = pd.concat([df.iloc[:,df.columns != i],pd.DataFrame(y_full)],axis = 1 )
    df_0 = SimpleImputer(missing_values=np.nan,strategy ="constant",fill_value = 0).fit_transform(df)

    ytrain = fillc[fillc.notnull()]
    ytest = fillc[fillc.isnull()]
    xtrain = df_0[ytrain.index,:]
    xtest = df_0[ytest.index,:]

    rfc = RandomForestRegressor(n_estimators=100)
    ffc =  rfc.fit(xtrain,ytrain)
    ypredict = rfc.predict(xtest)
    #将填好的特征返回到原来的原始特征矩阵中
    x_missing_reg.loc[x_missing_reg.iloc[:,i].isnull(),i] = ypredict

In [43]:
from sklearn.model_selection import cross_val_score
X = [x_full,x_missing_mean,x_missing_0,x_missing_reg]
mse=[]
for i in X:
    estimator = RandomForestRegressor(random_state=0,n_estimators=50)
    scores = cross_val_score(estimator,i,y_full,scoring = 'neg_mean_squared_error',cv=5).mean()
    mse.append(scores* -1)
    print(mse)

[nan]
[nan, 41.02723215585322]
[nan, 41.02723215585322, 48.13800805598913]
[nan, 41.02723215585322, 48.13800805598913, nan]


In [40]:
#mse
[*zip(['x_full','x_missing_mean','x_missing_0','x_missing_reg'],mse)]

[('x_full', nan),
 ('x_missing_mean', 40.84405476955929),
 ('x_missing_0', 49.50657028893417),
 ('x_missing_reg', nan)]

In [ ]:
from sklearn.model_selection import cross_val_score
X = [x_full,x_missing_mean,x_missing_0,x_missing_reg]
X

In [15]:
df = x_missing_reg
df

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,0.371307,18.00,6.3306,0.16,0.538000,6.44598,65.200,4.090000,1.00,296.00,17.621,389.5116,4.98
1,0.027310,0.00,7.5555,0.00,0.469000,6.10396,78.900,4.967100,2.00,275.46,18.369,396.9000,9.14
2,0.027290,46.12,7.0700,0.00,0.455410,7.18500,61.100,5.387765,2.00,242.00,17.604,388.5375,NaN
3,0.083621,25.68,3.3414,0.00,0.458000,6.96625,45.800,4.258761,3.93,222.00,18.700,393.6630,NaN
4,0.081062,0.00,2.1800,0.00,0.475881,7.14700,52.886,4.402685,3.58,231.88,18.700,394.3353,5.33
...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.508433,0.55,11.4698,0.00,0.573000,6.31512,69.100,2.829191,1.00,277.31,21.000,391.3595,9.67
502,0.045270,0.00,11.9300,0.00,0.573000,6.12000,76.700,2.287500,1.00,273.00,19.475,396.9000,9.08
503,0.527556,3.02,11.9300,0.29,0.573000,6.97600,91.000,2.796249,2.94,256.52,21.000,389.9534,5.64
504,0.109590,0.00,11.9300,0.11,0.573000,6.37046,89.300,2.806328,1.00,274.02,21.000,393.4500,6.48


In [16]:
fillc=df.iloc[:,6]
fillc

0      65.200
1      78.900
2      61.100
3      45.800
4      52.886
        ...  
501    69.100
502    76.700
503    91.000
504    89.300
505    83.897
Name: 6, Length: 506, dtype: float64

In [17]:
pd.concat([df.iloc[:,df.columns !=6],pd.DataFrame(y_full)],axis = 1)

,0,1,2,3,4,5,7,8,9,10,11,12,0
0,0.371307,18.00,6.3306,0.16,0.538000,6.44598,4.090000,1.00,296.00,17.621,389.5116,4.98,24.0
1,0.027310,0.00,7.5555,0.00,0.469000,6.10396,4.967100,2.00,275.46,18.369,396.9000,9.14,21.6
2,0.027290,46.12,7.0700,0.00,0.455410,7.18500,5.387765,2.00,242.00,17.604,388.5375,NaN,34.7
3,0.083621,25.68,3.3414,0.00,0.458000,6.96625,4.258761,3.93,222.00,18.700,393.6630,NaN,33.4
4,0.081062,0.00,2.1800,0.00,0.475881,7.14700,4.402685,3.58,231.88,18.700,394.3353,5.33,36.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.508433,0.55,11.4698,0.00,0.573000,6.31512,2.829191,1.00,277.31,21.000,391.3595,9.67,22.4
502,0.045270,0.00,11.9300,0.00,0.573000,6.12000,2.287500,1.00,273.00,19.475,396.9000,9.08,20.6
503,0.527556,3.02,11.9300,0.29,0.573000,6.97600,2.796249,2.94,256.52,21.000,389.9534,5.64,23.9
504,0.109590,0.00,11.9300,0.11,0.573000,6.37046,2.806328,1.00,274.02,21.000,393.4500,6.48,22.0


In [18]:
df_0 = SimpleImputer(missing_values=np.nan,strategy ="constant",fill_value = 0).fit_transform(df)

In [ ]:
ytrain = fillc[fillc.notnull()]
ytest = fillc[fillc.isnull()]
xtrain = df_0[ytrain.index,:]
xtest = df_0[ytest.index,:]

rfc = RandomForestRegressor(n_estimators=100)
ffc =  rfc.fit(xtrain,ytrain)
ypredict = rfc.predict(xtest)
x_missing_reg.loc[x_missing_reg.iloc[:,6].isnull(),6] = ypredict

In [ ]:
ypredict